In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# 데이터 불러오기
data = pd.read_csv('C:/Users/임진성/Desktop/LSTM_V1/your_data.csv')

# URL과 type 열 분리
urls = data['url']
labels = data['type']

# 라벨 인코딩
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)

# URL 인코딩
tokenizer = Tokenizer(char_level=True)  # 각 문자를 토큰으로 인식
tokenizer.fit_on_texts(urls)
X = tokenizer.texts_to_sequences(urls)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# tf.data.Dataset 객체 생성
train_dataset = tf.data.Dataset.from_generator(
    lambda: ((np.array(seq, dtype=np.int32), label) for seq, label in zip(X_train, y_train)),
    output_signature=(
        tf.TensorSpec(shape=(None,), dtype=tf.int32),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
).padded_batch(256, padded_shapes=([None], []))

test_dataset = tf.data.Dataset.from_generator(
    lambda: ((np.array(seq, dtype=np.int32), label) for seq, label in zip(X_test, y_test)),
    output_signature=(
        tf.TensorSpec(shape=(None,), dtype=tf.int32),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
).padded_batch(256, padded_shapes=([None], []))

# 모델 구성
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=64),
    LSTM(64, return_sequences=True),
    LSTM(32, return_sequences=False),
    Dense(len(np.unique(y)), activation='softmax')
])

# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset)

# 모델 평가
loss, accuracy = model.evaluate(test_dataset)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


Epoch 1/10
2035/2035 [==============================] - 140s 67ms/step - loss: 0.9887 - accuracy: 0.6561 - val_loss: 0.9862 - val_accuracy: 0.6586
Epoch 2/10
2035/2035 [==============================] - 126s 62ms/step - loss: 0.9885 - accuracy: 0.6571 - val_loss: 0.9860 - val_accuracy: 0.6586
Epoch 3/10
2035/2035 [==============================] - 124s 61ms/step - loss: 0.9879 - accuracy: 0.6571 - val_loss: 0.9852 - val_accuracy: 0.6588
Epoch 4/10
2035/2035 [==============================] - 137s 67ms/step - loss: 0.7768 - accuracy: 0.7145 - val_loss: 0.5488 - val_accuracy: 0.7882
Epoch 5/10
2035/2035 [==============================] - 140s 69ms/step - loss: 0.5073 - accuracy: 0.8143 - val_loss: 0.4573 - val_accuracy: 0.8448
Epoch 6/10
2035/2035 [==============================] - 140s 69ms/step - loss: 0.4484 - accuracy: 0.8447 - val_loss: 0.4474 - val_accuracy: 0.8449
Epoch 7/10
2035/2035 [==============================] - 135s 66ms/step - loss: 0.3860 - accuracy: 0.8659 - val_loss: 0